In [35]:
import platform
from pathlib import Path
import yaml
import pandas as pd

assert platform.node() == 'habilis', "Code should run on habilis"
""" Global Paths """
source_leaderboards = Path('/home/nhamilakis/workspace/zerospeech/core/leaderboards/data/tasks/abx/abxLS/parted')
new_scores_location = Path('/scratch2/mhallap/zerospeech/evaluations/abx_evaluations/2022_evals/dtw_runs/v095/results')
dest_dir = Path('/home/nhamilakis/workspace/zerospeech/core/leaderboards/data/tasks/abx/abxLS-v2/parted')


In [36]:
""" Old Leaderboard definitions """
from typing import Literal
from dataclasses import dataclass
from typing import Dict, Any, List
import yaml

@dataclass
class ABXLSScore:
    subset: Literal['dev-clean', 'dev-other', 'test-clean', 'test-other']
    granularity: Literal['triphone', 'phoneme']
    speaker_mode: Literal['across', 'within']
    context_mode: Literal['within', 'any']
    score: float
    pooling: str
    seed: int
    
    def to_dict(self) -> Dict:
        return dict(
            subset=self.subset,
            granularity=self.granularity,
            speaker_mode=self.speaker_mode,
            context_mode=self.context_mode,
            score=self.score,
            pooling=self.pooling,
            seed=self.seed
        )

In [37]:
"""Load leaderboard entries from the old abxLS without the scores section and indexed by their submission_id"""
from typing import Dict


def fomat_scores(sc: Dict):
    """ Format scores """
    return [
        # dev-clean within/across speaker
        ABXLSScore(
            subset='dev-clean',
            speaker_mode='within',
            context_mode='within',
            granularity='triphone',
            score=sc['clean']['dev']['within'],
            pooling='none',
            seed='none'
        ),
        ABXLSScore(
            subset='dev-clean',
            speaker_mode='across',
            context_mode='within',
            granularity='triphone',
            score=sc['clean']['dev']['across'],
            pooling='none',
            seed='none'
        ),
        # dev-other within/across speaker
        ABXLSScore(
            subset='dev-other',
            speaker_mode='within',
            context_mode='within',
            granularity='triphone',
            score=sc['other']['dev']['within'],
            pooling='none',
            seed='none'
        ),
        ABXLSScore(
            subset='dev-other',
            speaker_mode='across',
            context_mode='within',
            granularity='triphone',
            score=sc['other']['dev']['across'],
            pooling='none',
            seed='none'
        ),
        # test-clean within across speaker
        ABXLSScore(
            subset='test-clean',
            speaker_mode='within',
            context_mode='within',
            granularity='triphone',
            score=sc['clean']['test']['within'],
            pooling='none',
            seed='none'
        ),
        ABXLSScore(
            subset='test-clean',
            speaker_mode='across',
            context_mode='within',
            granularity='triphone',
            score=sc['clean']['test']['across'],
            pooling='none',
            seed='none'
        ),
        # test-other within/across speaker
        ABXLSScore(
            subset='test-other',
            speaker_mode='within',
            context_mode='within',
            granularity='triphone',
            score=sc['other']['test']['within'],
            pooling='none',
            seed='none'
        ),
        ABXLSScore(
            subset='test-other',
            speaker_mode='across',
            context_mode='within',
            granularity='triphone',
            score=sc['other']['test']['across'],
            pooling='none',
            seed='none'
        )
    ]

old_leaderboard = {}
for file in source_leaderboards.glob('*.yaml'):
    with file.open() as fp:
        data = yaml.load(fp, Loader=yaml.FullLoader)

    submission_id = data.get('submission_id', None)
    if submission_id is not None:
        data['scores'] = fomat_scores(data['scores'])        
        old_leaderboard[submission_id] = data


In [38]:
import pandas as pd


def clean_scores(df: pd.DataFrame, subset: str):
    # remove unused data
    del df["path_data"]
    del df["item-file"]
    del df["dataset"]
    del df["sub-dataset"]
    # add granularity & subset
    df['granularity'] = "phoneme"
    df['subset'] = subset
    # rename columns to be more clean
    df = df.rename(
        columns={'abx-s-condition': 'speaker_mode', 'abx-c-condition': 'context_mode'})
    # reorder columns
    cols = ['subset', 'speaker_mode', 'context_mode',
            'granularity', 'score', 'pooling', 'seed']
    df = df[cols]
    return df


def load_submission(location: Path, sub_dir: str):
    """ Load a submission from Marks evaluation format """
    location = location / sub_dir
    scores_list = []

    for _set in location.iterdir():
        if _set.is_dir():
            df = pd.read_csv(_set / 'ABX_scores.csv')
            df = clean_scores(df, subset=_set.name)
            scores_list.append(df)

    df = pd.concat(scores_list)
    df.reset_index(inplace=True)
    del df['index']
    return [ABXLSScore(**i) for i in df.to_dict(orient='records')]


def load_all_submissions(location: Path, sub_dir: str):
    """ Load all new scores from location """
    new_scores = {}
    for item_location in location.iterdir():
        if item_location.is_dir():
            sub_id = item_location.name
            new_scores[sub_id] = load_submission(item_location, sub_dir)
    return new_scores


subs_batch1 = Path(
    '/scratch2/mhallap/zerospeech/evaluations/abx_evaluations/2022_evals/dtw_runs/v095/results/')
subs_batch2 = Path(
    '/scratch2/mhallap/zerospeech/evaluations/abx_evaluations/2022_evals/dtw_runs/v097')

new_scores = load_all_submissions(subs_batch1, sub_dir='phonetic')
new_scores.update(load_all_submissions(subs_batch2, sub_dir='input/phonetic'))


In [39]:
""" Build New leaderboard Objects """


def dump_scores(submission_id: str, score_list: List[ABXLSScore]):
    """ Dump scores link to old leaderbord using submission_id to gather submission metadata """
    old = old_leaderboard.get(submission_id, None)

    if old is None:
        print(f'Not found {submission_id} !!!')
        return
    
    # add new scores to old ones
    score_list.extend(old['scores'])
    # convert to dic & write as yaml
    old['scores'] = [i if isinstance(i, dict) else i.to_dict() for i in score_list]
    with (dest_dir / f"{submission_id}.yaml").open('w') as fp:
        yaml.dump(old, fp)


for sub_id, n_sc in new_scores.items():
    dump_scores(sub_id, n_sc)

Not found 20211112161645_harwath !!!
Not found 20211119235733_jaeyeonkim99 !!!
Not found saurabh_bhati !!!
Not found zr2021-baseline-random !!!


In [41]:
""" Merge all entries """
from datetime import datetime
import json

leaderboard_file = Path('/home/nhamilakis/workspace/zerospeech/core/leaderboards') / 'data/tasks/abx/abxLS-v2/abxLS-v2.json'

data_list = []
for f in dest_dir.glob('*.yaml'):
    with f.open() as fp:
        data_list.append(yaml.load(fp, Loader=yaml.FullLoader))


with leaderboard_file.open('w') as fp:
    json.dump(
        dict(
            updatedOn=datetime.now().isoformat(),
            data=data_list
        ), fp, indent=4
    )        

In [42]:
""" A quick script to fix score formatting """
from datetime import datetime
import json

from collections import defaultdict



def fix_score(dt):
    def nested_dict():
        return defaultdict(nested_dict)
    
    new_scores = nested_dict()
    for entry in dt:
        subset_type, subset_name = entry['subset'].split('-')
        new_scores[entry['granularity']][entry['context_mode']][subset_name][entry['speaker_mode']][subset_type] = entry
    
    return new_scores

# load leaderboard
with leaderboard_file.open() as fp:
    data = json.load(fp)
    

# fix scores
entries = data.get('data')
new_entries = []
for entry in entries:
    entry['scores'] = fix_score(entry['scores'])
    new_entries.append(entry)
    
# save leaderboard
with leaderboard_file.with_suffix(".new.json").open('w') as fp:
    json.dump(dict(
       updatedOn=datetime.now().isoformat(),
       data=new_entries
    ), fp, indent=4)